### gmarket
    - 베스트 상품 200개 수집

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# 1. URL

In [6]:
url = "http://corners.gmarket.co.kr/Bestsellers"
url

'http://corners.gmarket.co.kr/Bestsellers'

In [3]:
# 2. requests > response : HTML(str)

In [10]:
response = requests.get(url)
response

<Response [200]>

In [11]:
response.text[:500]

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\r\n<html lang="ko">\r\n<head>\r\n\t<title>G마켓 - G마켓 베스트</title>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n\t<meta http-equiv="content-type" content="text/html; charset=utf-8" />\r\n\t<meta http-equiv="content-style-type" content="text/css" />\r\n\t<meta http-equiv="content-script-type" content="text/javascript" />\r\n\t<meta http-equiv="imagetoolbar" content="no" />\r\n\t<meta id="viewport" name="'

In [4]:
# 3. HTML(str) > BeautifulSoup Object

In [17]:
dom = BeautifulSoup(response.text, "html.parser")
type(dom)

bs4.BeautifulSoup

In [20]:
# select() : 엘리먼트 여러개 선택
# select_one() : 엘리먼트 한개 선택
elements = dom.select("#gBestWrap > div > div:nth-child(5) > div > ul > li")
len(elements)

200

In [24]:
elements[0]

<li class="first">
<p class="no1" id="no1">1</p>
<div class="thumb">
<a href="http://item.gmarket.co.kr/Item?goodscode=1798110680&amp;ver=637952284067683932" onclick="pdsClickLog('200000680', 'Item', {'ASN': 1, 'goodsCode': '1798110680'});"><img alt="[한상궁김치]한상궁김치 국산 겉절이 2kg 출고당일생산" class="lazy" data-original="//gdimg.gmarket.co.kr/1798110680/still/300?ver=2022080416" height="210" src="//image.gmarket.co.kr/challenge/neo_image/no_image.gif" width="210"/></a>
</div>
<!--div class="goods-view">
									<a href="http://minishop.gmarket.co.kr/hansanggungfood"><span class="view">판매자 다른상품 보기</span>맛있는한상궁</a>
								</div-->
<a class="itemname" href="http://item.gmarket.co.kr/Item?goodscode=1798110680&amp;ver=637952284067683932" onclick="pdsClickLog('200000680', 'Item', {'ASN': 1, 'goodsCode': '1798110680'});">[한상궁김치]한상궁김치 국산 겉절이 2kg 출고당일생산</a>
<div class="item_price">
<div class="o-price"><span><span>23,400원</span></span></div>
<div class="s-price">
<strong><span><span>11,000원</span></span>

In [5]:
# 4. BeautifulSoup Object > css-selector > DataFrame

In [35]:
keywords = []
for element in elements:
    keyword = element.select_one(".itemname").text
    
    keywords.append(keyword)
    
print(keywords[:5])

['[한상궁김치]한상궁김치 국산 겉절이 2kg 출고당일생산', '[광동]아이스웨일 레몬라임 탄산수 500ml 40pet/음료/음료수', '[바디스토리]남성 누디5매 냉감 쿨 드로즈 남자 아이스 사각 팬티 / 무료반품+10%+15%할인', '(쌀가게) (광주신세계)[주문폭증으로 순차적 출고 예정] [2021년 햅쌀] 쌀가게  수향미 (특등급 )10KG', '[이지바이](무료반품)균일6900원/남녀 쿨론티셔츠/반바지/추리닝']


In [28]:
keywords = [element.select_one(".itemname").text for element in elements]
print(keywords[:5])

['[한상궁김치]한상궁김치 국산 겉절이 2kg 출고당일생산', '[광동]아이스웨일 레몬라임 탄산수 500ml 40pet/음료/음료수', '[바디스토리]남성 누디5매 냉감 쿨 드로즈 남자 아이스 사각 팬티 / 무료반품+10%+15%할인', '(쌀가게) (광주신세계)[주문폭증으로 순차적 출고 예정] [2021년 햅쌀] 쌀가게  수향미 (특등급 )10KG', '[이지바이](무료반품)균일6900원/남녀 쿨론티셔츠/반바지/추리닝']


In [55]:
elemente = elements[0]

data = {
    "title": element.select_one(".itemname").text,
    "o_price": element.select_one(".o-price").text, 
    "s_price": element.select_one(".s-price > strong").text, 
    "img": element.select_one("img").get("data-original"),
}
data

{'title': '김치메밀전병900gx2봉1.8kg/만두 간식 메밀 2봉/HACCP/',
 'o_price': '19,900원',
 's_price': '12,500원',
 'img': '//gdimg.gmarket.co.kr/1521852173/still/300?ver=2022080416'}

In [48]:
items = []
for element in elements:
    data = {
        "title": element.select_one(".itemname").text,
        "o_price": element.select_one(".o-price").text, 
        "s_price": element.select_one(".s-price > strong").text, 
        "img": "http:" + element.select_one("img").get("data-original"),
    }
    items.append(data)
    
print(items[0])

{'title': '[한상궁김치]한상궁김치 국산 겉절이 2kg 출고당일생산', 'o_price': '23,400원', 's_price': '11,000원', 'img': 'http://gdimg.gmarket.co.kr/1798110680/still/300?ver=2022080416'}


In [44]:
df = pd.DataFrame(items)
df.tail(2)

,title,o_price,s_price,img
198,주니어김영사 놓지 마 어휘 한자어 1 2 3 4 번 전4권 세트,"81,000원","55,200원",http://gdimg.gmarket.co.kr/2298871614/still/30...
199,김치메밀전병900gx2봉1.8kg/만두 간식 메밀 2봉/HACCP/,"19,900원","12,500원",http://gdimg.gmarket.co.kr/1521852173/still/30...


In [45]:
# summary
# 웹 페이지의 종류
# - 동적페이지 : URL 변경 없이 웹 페이지 데이터 변경 : JSON : API
# - 정적페이지 : URL 변경 해서 웹 페이지 데이터 변경 : HTML : BeatifulSoup(css selector)

In [46]:
# 실습
# naver api : 검색어 트랜드
# zigbang : 원룸 데이터 수집
# daum exchange : headers(user-agent, referer)
# html, css-selector
# naver relational keyword
# gmarket items

In [ ]:
# crawling images
# crawling selenium
# xpath
# python web crawling f.w. : scrapy